In [16]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import model_from_json
from keras import metrics
from keras.preprocessing.image import ImageDataGenerator

## Reimplement the precision & recall metrics, as they were removed from the later versions of keras.

In [2]:
import keras.backend as K
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

# Real vs Spam Image Classifier

## Defining the model and it's architecture.

In [3]:
classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (128, 128, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (10, 10)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.3))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

## Compling the model.
##### Selecing the optimizer, the loss function, and the metrics we want to measure the performance against.

In [4]:
classifier.compile(optimizer = 'adamax', loss = 'binary_crossentropy', metrics = ['accuracy', metrics.binary_accuracy, precision , recall])

## Loading the training and testing datasets and performing transformations.

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('Keras/train_irma',
target_size = (128, 128),
batch_size = 15,
class_mode = 'binary')

test_set = test_datagen.flow_from_directory('Keras/test_irma',
target_size = (128, 128),
batch_size = 32,
class_mode = 'binary')

Found 643 images belonging to 2 classes.
Found 36 images belonging to 2 classes.


## Training the model.

In [6]:
classifier.fit_generator(training_set,
epochs = 25,
validation_data = test_set, workers=4)

Epoch 1/25
43/43 [==============================] - 7s 154ms/step - loss: 0.6102 - acc: 0.7807 - binary_accuracy: 0.7807 - precision: 0.1527 - recall: 0.1442 - val_loss: 1.0768 - val_acc: 0.4722 - val_binary_accuracy: 0.4722 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/25
43/43 [==============================] - 6s 135ms/step - loss: 0.4864 - acc: 0.8106 - binary_accuracy: 0.8106 - precision: 0.3360 - recall: 0.1864 - val_loss: 0.8058 - val_acc: 0.5833 - val_binary_accuracy: 0.5833 - val_precision: 0.8222 - val_recall: 0.2647
Epoch 3/25
43/43 [==============================] - 6s 138ms/step - loss: 0.4138 - acc: 0.8481 - binary_accuracy: 0.8481 - precision: 0.4631 - recall: 0.3007 - val_loss: 0.4906 - val_acc: 0.7500 - val_binary_accuracy: 0.7500 - val_precision: 0.6667 - val_recall: 0.7407
Epoch 4/25
43/43 [==============================] - 6s 136ms/step - loss: 0.3849 - acc: 0.8494 - binary_accuracy: 0.8494 - precision: 0.3886 - recall: 0.3705 - val_loss: 0.6510 - val

## Calculating model's accuracy.

In [11]:
score = classifier.evaluate_generator(training_set,
                             workers=4)

In [23]:
print("%s: %.2f%%" % (classifier.metrics_names[0], score[0]*100))
print("%s: %.2f%%" % (classifier.metrics_names[1], score[1]*100))
print("%s: %.2f%%" % (classifier.metrics_names[2], score[2]*100))
print("%s: %.2f%%" % (classifier.metrics_names[3], score[3]*100))
print("%s: %.2f%%" % (classifier.metrics_names[4], score[4]*100))

loss: 9.99%
acc: 96.73%
binary_accuracy: 96.73%
precision: 91.21%
recall: 85.76%


## Saving the model to secondary storage.
##### We are doing this to avoid retraining the model again.

In [17]:
# serialize model to JSON
model_json = classifier.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
classifier.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


## Loading the model when needed.
##### We need to recompile the model after loading the weights.

In [27]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [28]:
loaded_model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy', metrics.binary_accuracy, precision , recall])
score = loaded_model.evaluate_generator(training_set)

In [29]:
print("%s: %.2f%%" % (loaded_model.metrics_names[0], score[0]*100))
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
print("%s: %.2f%%" % (loaded_model.metrics_names[2], score[2]*100))
print("%s: %.2f%%" % (loaded_model.metrics_names[3], score[3]*100))
print("%s: %.2f%%" % (loaded_model.metrics_names[4], score[4]*100))

loss: 8.78%
acc: 96.58%
binary_accuracy: 96.58%
precision: 90.67%
recall: 81.88%


# Flooding vs Wind vs Destruction Image Classifier.
##### Only "Real" images go through this classifier.

## Defining the model and it's architecture.
##### NOT DONE, YET!

In [21]:
classifier2 = Sequential()
# Layers should be added here.

## Loading the training and testing datasets and performing transformations.

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('Keras2/train',
target_size = (128, 128),
batch_size = 15,
class_mode='categorical')

test_set = test_datagen.flow_from_directory('Keras2/test',
target_size = (128, 128),
batch_size = 3,
class_mode = 'categorical')

## Training the model.

In [ ]:
classifier2.fit_generator(training_set,
epochs = 25,
validation_data = test_set, workers=4)

## Testing the model against an example.

In [ ]:
import numpy as np
from keras.preprocessing import image
img_path = '/home/saed/Desktop/Research/Twitter/Images Classification/images/2bO4EEqOGPwAFFrC.jpg'
test_image = image.load_img(img_path, target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image/255.0, axis = 0)
result = classifier.predict_classes(test_image)